In [19]:
#https://data.gov.tw/dataset/9176
#自動氣象站-氣象觀測資料
#https://opendata.cwb.gov.tw/fileapi/v1/opendataapi/O-A0001-001?Authorization=rdec-key-123-45678-011121314&format=JSON
#urllib3說明
#https://urllib3.readthedocs.io/

#ssl解決方案
#https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
    
import urllib3
import certifi
import json

#使用ultlib3, certifi, json下載資料和解析資料
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',ca_certs=certifi.where())
r = http.request('GET', 'https://opendata.cwb.gov.tw/fileapi/v1/opendataapi/O-A0001-001?Authorization=rdec-key-123-45678-011121314&format=JSON')

if r.status == 200:
    downloadData=json.loads(r.data.decode('utf-8'))
else:
    print('下載錯誤')

#轉換成要使用的資料
usefulData = [];
opendata = downloadData['cwbopendata']['location']
for location in opendata:
    oneLocation = {}
    oneLocation['縣市']=location['parameter'][0]['parameterValue']
    oneLocation['區域']=location['parameter'][2]['parameterValue']
    oneLocation['時間']=location['time']['obsTime']
    oneLocation['溫度']=float(location['weatherElement'][3]['elementValue']['value'])
    usefulData.append(oneLocation)


In [23]:
import numpy as np
import pandas as pd
temperature = pd.DataFrame(usefulData)
temperature.head(20)

,縣市,區域,時間,溫度
0,臺北市,南港區,2020-04-16T10:00:00+08:00,25.3
1,新竹縣,竹北市,2020-04-16T10:00:00+08:00,24.2
2,彰化縣,鹿港鎮,2020-04-16T10:00:00+08:00,23.9
3,雲林縣,臺西鄉,2020-04-16T10:00:00+08:00,24.0
4,臺南市,七股區,2020-04-16T10:00:00+08:00,22.9
5,臺東縣,成功鎮,2020-04-16T10:00:00+08:00,24.7
6,南投縣,南投市,2020-04-16T10:00:00+08:00,25.7
7,雲林縣,林內鄉,2020-04-16T10:00:00+08:00,23.0
8,高雄市,茂林區,2020-04-16T10:00:00+08:00,27.4
9,連江縣,莒光鄉,2020-04-16T10:00:00+08:00,18.9
